In [1]:
import collections
import bisect
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.signal as signal

from sklearn.neighbors import KernelDensity
# from sklearn.grid_search import GridSearchCV
# from sklearn.cross_validation import LeaveOneOut
from sklearn.mixture import GaussianMixture
# from sklearn.cross_validation import KFold




sns.set_context("talk", font_scale=1, rc={"lines.linewidth": 2.0, 'lines.markersize': 5})
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})

tw = 1.5
sns.set_style({"xtick.major.size": 6, "ytick.major.size": 6,
               "xtick.minor.size": 4, "ytick.minor.size": 4,
               'axes.labelsize': 24,
               'xtick.major.width': tw, 'xtick.minor.width': tw,
               'ytick.major.width': tw, 'ytick.minor.width': tw})

mpl.rc('xtick', labelsize=18) 
mpl.rc('ytick', labelsize=18)
mpl.rc('axes', linewidth=1.75)
plt.gcf().subplots_adjust(bottom=0.15)
sns.set_style({'axes.labelsize': 24})


%matplotlib inline
# This enables high res graphics inline
%config InlineBackend.figure_format = 'svg'

In this Jupyter notebook, we will fit the flow cytometry results from sfYFP with KDEs, assign individual cells to peaks, then determine the fractions that are 'on' and 'off' in each well. These fractions will be combined with the results from the second multi-dilution T7 differentiation experiment to generate the flow cytometry portion of figure 4. Thoug 0uM IPTG was included as a concentration during these experiments and in the peak detection/assignment, we will not use it in generating figures. Code below is modified from code kindly provided by Andy Halleran on the Github repositiory: https://github.com/andyhalleran/flow_tools/blob/. Functions with 'RW' have been modified from the original code available on this repository, or are new functions.

In [2]:
def get_values(file_directory, channel='GFP/FITC-A',bg_add=0):
    ''' Reads in the values from a specific channel for a given flow file.
    Defaults to taking GFP/FITC-A.'''
    flow_data = pd.read_csv(file_directory)
    flow_data_gfp_values = np.log10(flow_data[channel].values + bg_add)
    return flow_data_gfp_values


In [3]:
def make_df(file_directory, channel='GFP/FITC-A'):
    '''Creates a dataframe from the given file directory. Reads in all csvs, 
    extracts the data from the channel of interest (defaults to GFP/FITC-A), 
    and returns one dataframe.'''
    all_files = glob.glob(file_directory)
    all_files.sort()
    
    all_data = []
    for file in all_files:
        data = get_values(file, channel)
        all_data.append(data)
    
    labels = []
    for i in range(0, len(all_files)):
        mini_label = str(all_files[i].split('.')[-2][-3:])
        label = [mini_label]*len(all_data[i])
        labels.append(label)
    
    flat_all_data = [item for sublist in all_data for item in sublist]
    flat_labels = [item for sublist in labels for item in sublist]
    
    df = pd.DataFrame(dict(well=flat_labels, log10GFP=flat_all_data))
    return df
        

In [4]:
def get_peak_locations_from_KDE_fit(data):
    ''' Performs a KDE fit and then uses scipy.signal.find_peaks_cwt to get peaks.
        The KDE bandwith parameter is critical, and 0.25 has worked well in the past.
        If it feels like you are missing many peak calls, decrease the bandwith. If it feels
        like you are having too many peak calls, increase the bandwith. 
        
        Don't change the bandwith without good reason, it took awhile to decide on 0.25. '''
    
    kde = KernelDensity(bandwidth=0.25, kernel='gaussian')
    kde.fit(data[:, None]);

    x_range = np.linspace(0, 6, 1200)
    kde_estimates = np.exp(kde.score_samples(x_range[:, None]))

    #Use the SciPy function to get the KDE peaks
    peaks = signal.find_peaks_cwt(kde_estimates, np.arange(30, 200), min_snr=1)

    means_init = []
    
    for peak in peaks:
        means_init.append(x_range[peak])
    
    return means_init



In [5]:
def fit_GMM_KDE_RW(data, peaks, threshold = 0.01): 
    """Generate a Gaussian mixture model from the output
    of a Gaussian Kernel Density Estimation. 
    Outputs the mean of the on peak, fraction on, mean of the off peak, 
    and fraction off. This version of the code assumes all cells not in the on peak are off!
    This is obviously only a good assumption for uni/bimodal data. If you have multimodal data,
    do not use this code."""
    
    data = data.reshape(len(data), 1)
    
    peaks = np.array(peaks).reshape(len(peaks), 1)
    opt_gmm = GaussianMixture(n_components = len(peaks) , means_init = peaks).fit(data)  
    
    labels = opt_gmm.predict(data)
    labels = np.ravel(labels.reshape(len(labels), 1))

    means = opt_gmm.means_

    df = pd.DataFrame({'gfp': np.ravel(data), 'distribution': labels})
    
    df.head(10)
    counts = []
    means = []
    
    
    
    for i in range(0, len(peaks)):
        df_distro = df.loc[df['distribution']==i]
        counts.append(len(df_distro))
        means.append(np.mean(df_distro['gfp'].values))

    print('counts = ', counts)
    print('means = ', means)
    total = len(df)
    
    fractions = np.array(counts)/total
    
    ##Initializing corrected lists of means and fractions of subpopulations
    GMM_accepted_means = []
    
    GMM_corrected_fractions = []
    
    for i in range(0, len(fractions)):
        if fractions[i] > threshold: 
            GMM_accepted_means.append(means[i])
            GMM_corrected_fractions.append(fractions[i])    
    
    return GMM_accepted_means, GMM_corrected_fractions

In [6]:
def fit_GMM_KDE_wrapper_RW(data):
    """Wrapper function to get both the peaks from a KDE fit, and then 
    from the Gaussian mixture model. Returns the mean of the broken cells, 
    and the fraction of broken cells."""
    
    peak_locations= get_peak_locations_from_KDE_fit(data)
    
    means, fractions = fit_GMM_KDE_RW(data, peak_locations, threshold = 0.01)
    
    return means, fractions

In [7]:
def GMM_method_RW(df_, wells):
    '''Wrapper function for the the entire generation of the final output df. 
    Takes the input dataframe and a list of all wells you want to perform GMM fitting on.
    '''
    means_df = []
    fractions_df = []
    wells_df = []
    
    for well in wells:
        data = df_.loc[df_['well'] == well]
        
        YFP = data['log10GFP'].values
        if len(YFP)<1000:
            print('error: well ' + well + 'has too few cells')
        else:
            means, fractions = fit_GMM_KDE_wrapper_RW(YFP)

            means_df = means_df + means
            fractions_df = fractions_df + fractions
            wells_df = wells_df + len(means)*[well,]
        

    plt_df = pd.DataFrame({'mean': means_df, 'fraction' : fractions_df,
                           'well': wells_df})
    
    return plt_df

In [8]:
def plot_flow_peaks(ax,df,wells,conditions,colors,offsets=np.array([-0.2,-0.1,0,0.1,0.2]),
                   xticks=np.arange(10)):
    x_plot = []
    y_plot = []
    sizes_plot = []
    colors_plot = []
    for i, well in enumerate(wells):
        well_peaks = df.loc[df.well==well]
        for j in range(len(well_peaks)):
            condition = well_peaks.iloc[j]['condition']
            loc = np.argwhere(conditions==condition).flatten()[0]
            x_plot += [well_peaks.iloc[j]['plate'] + offsets[loc],]
            y_plot += [well_peaks.iloc[j]['mean'],]
            sizes_plot += [well_peaks.iloc[j]['fraction']*100,] 
            colors_plot += [colors[loc],]
            
    ax.set_xticks(xticks)
    ax.scatter(x_plot, y_plot, s=sizes_plot, c=np.array(colors_plot),alpha=0.5)
    return ax

In [9]:
def get_frac_on(df, w, on_thresh=2.5):
    """
    Takes dataframe of all peaks with peak location 'mean' comprising fraction 'fraction'
    for wells 'w', and returns dataframe with fraction on, mean value of the on fraction for each well
    """
    wells_df = []
    frac_on_df = []
    on_mean_df = []
    for i, well in enumerate(w):
        peaks = df.loc[df.well==well]
        fracs = df.loc[(df.well==well) & (df['mean']>on_thresh),'fraction'].values
        means = df.loc[(df.well==well) & (df['mean']>on_thresh),'mean'].values
        if fracs.sum() == 0:
            frac_on = 0
            weighted_mean_on = np.nan
        else:
            weighted_mean_on = np.log10((np.power(10,means)*fracs/fracs.sum()).sum())
            frac_on = fracs.sum()
        wells_df += [well,]
        frac_on_df += [frac_on,]
        on_mean_df += [weighted_mean_on,]
        
    frac_df = pd.DataFrame({'mean': on_mean_df, 'fraction' : frac_on_df,
                       'well': wells_df})  
    return frac_df                


In [10]:
def select_wells(df, wells):
    """
    Returns dataframe with only wells in 'wells'
    """
    for i, well in enumerate(wells):

        if i == 0:
            df_select = df.loc[df.well==well,:]
        else:
            df_add = df.loc[df.well==well,:]
            df_select = pd.concat([df_select, df_add])
    return df_select

def add_metadata(df, metadata_fs, cols=['strain','sal','las','iptg','chlor','replicate']):
    """
    Returns dataframe with metadata added for cols
    """
    wells = df.well.unique()
    metadata = pd.read_csv(metadata_fs)
    df_return = df.copy()
    
    for i, col in enumerate(cols):
        df_return[col] = -1
    df_return['condition'] = 'blank'
    for i, well in enumerate(wells):
        for j, col in enumerate(cols):
            df_return.loc[df_return.well==well,col] = metadata.loc[metadata.well==well,col].values[0]
        if metadata.loc[metadata.well==well,'strain'].values[0] == 'ctrl':
            df_return.loc[df_return.well==well,'condition'] = 'naive'
        elif metadata.loc[metadata.well==well,'strain'].values[0] == 'stem':
            if metadata.loc[metadata.well==well,'chlor'].values[0] == '+':
                df_return.loc[df_return.well==well,'condition'] = \
                    str(metadata.loc[metadata.well==well,'sal'].values[0]) + ' sal' + ' +chlor'
            elif metadata.loc[metadata.well==well,'chlor'].values[0] == '-':
                df_return.loc[df_return.well==well,'condition'] = \
                    str(metadata.loc[metadata.well==well,'sal'].values[0]) + ' sal'
            else:
                print('somethin is wrong')
                
    return df_return
   
def select_add_metadata(df, metadata_fs, wells, cols=['strain','sal','las','iptg','chlor','replicate']):
    """
    Returns dataframe for wells in 'well' with metadata added
    """
    df = select_wells(df, wells)
    
    return add_metadata(df, metadata_fs, cols=cols)

In [11]:
# Initialize the list of wells that you have data files for and wish to fit with GMMs
wells = ['A01', 'B01', 'C01', 'D01', 'E01', 'F01', 'G01', 'H01',
         'A02', 'B02', 'C02', 'D02', 'E02', 'F02', 'G02', 'H02',
         'A03', 'B03', 'C03', 'D03', 'E03', 'F03', 'G03', 'H03',
         'A04', 'B04', 'C04', 'D04', 'E04', 'F04', 'G04', 'H04',
         'A05', 'B05', 'C05', 'D05', 'E05', 'F05', 'G05', 'H05',
         'A06', 'B06', 'C06', 'D06', 'E06', 'F06', 'G06', 'H06',
         'A07', 'B07', 'C07', 'D07', 'E07', 'F07', 'G07', 'H07',
         'A08', 'B08', 'C08', 'D08', 'E08', 'F08', 'G08', 'H08',
         'A09', 'B09', 'C09', 'D09', 'E09', 'F09', 'G09', 'H09',
         'A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10',
         'A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11']

wells2 = ['A01', 'B01', 'C01', 'D01', 'E01', 'F01', 'G01', 'H01',
         'A02', 'B02', 'C02', 'D02', 'E02', 'F02', 'G02', 'H02',
         'A06', 'B06', 'C06', 'D06', 'E06', 'F06', 'G06', 'H06',
         'A07', 'B07', 'C07', 'D07', 'E07', 'F07', 'G07', 'H07',
         'A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11']

In [12]:
#Read in the tidied data generated by "ADH_automatic_flow_gating_and_well_labeling.ipynb"
# The input for make_df is the directory that you want automatic fractions generated for. 
input_df_plate1 = make_df('./flow/plate1/tidy/*.csv',channel='GFP/FITC-A')
input_df_plate1 = input_df_plate1.loc[(0<input_df_plate1.log10GFP) & \
                                      (input_df_plate1.log10GFP<5),:]
input_df_plate1 = input_df_plate1.dropna()

input_df_plate2 = make_df('./flow/plate2/tidy/*.csv',channel='GFP/FITC-A')
input_df_plate2 = input_df_plate2.loc[(input_df_plate2.log10GFP>0) & \
                                      (input_df_plate2.log10GFP<5),:]
input_df_plate2 = input_df_plate2.dropna()

input_df_plate3 = make_df('./flow/plate3/tidy/*.csv',channel='GFP/FITC-A')
input_df_plate3 = input_df_plate3.loc[(input_df_plate3.log10GFP>0) & \
                                      (input_df_plate3.log10GFP<5),:]
input_df_plate3 = input_df_plate3.dropna()

input_df_plate4 = make_df('./flow/plate4/tidy/*.csv',channel='GFP/FITC-A')
input_df_plate4 = input_df_plate4.loc[(input_df_plate4.log10GFP>0) & \
                                      (input_df_plate4.log10GFP<5),:]
input_df_plate4 = input_df_plate4.dropna()

input_df_plate5 = make_df('./flow/plate5/tidy/*.csv',channel='GFP/FITC-A')
input_df_plate5 = input_df_plate5.loc[(input_df_plate5.log10GFP>0) & \
                                      (input_df_plate5.log10GFP<5),:]
input_df_plate5 = input_df_plate5.dropna()

input_df_plate6 = make_df('./flow/plate6/tidy/*.csv',channel='GFP/FITC-A')
input_df_plate6 = input_df_plate6.loc[(input_df_plate6.log10GFP>0) & \
                                      (input_df_plate6.log10GFP<5),:]
input_df_plate6 = input_df_plate6.dropna()

input_df_plate7 = make_df('./flow/plate7/tidy/*.csv',channel='GFP/FITC-A')
input_df_plate7 = input_df_plate7.loc[(input_df_plate7.log10GFP>0) & \
                                      (input_df_plate7.log10GFP<5),:]
input_df_plate7 = input_df_plate7.dropna()

input_df_plate8 = make_df('./flow/plate8/tidy/*.csv',channel='GFP/FITC-A')
input_df_plate8 = input_df_plate8.loc[(input_df_plate8.log10GFP>0) & \
                                      (input_df_plate8.log10GFP<5),:]
input_df_plate8 = input_df_plate8.dropna()
                          

/Users/rorywilliams/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: invalid value encountered in log10


In [13]:
# Generate data frames with peak means and fractions
plate1_peaks_RW = GMM_method_RW(input_df_plate1,wells)
plate2_peaks_RW = GMM_method_RW(input_df_plate2,wells)
plate3_peaks_RW = GMM_method_RW(input_df_plate3,wells)
plate4_peaks_RW = GMM_method_RW(input_df_plate4,wells)
plate5_peaks_RW = GMM_method_RW(input_df_plate5,wells)
plate6_peaks_RW = GMM_method_RW(input_df_plate6,wells2)
plate7_peaks_RW = GMM_method_RW(input_df_plate7,wells2)
plate8_peaks_RW = GMM_method_RW(input_df_plate8,wells2)


/Users/rorywilliams/anaconda/lib/python3.5/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


counts =  [473496]
means =  [1.8700392622765076]


/Users/rorywilliams/anaconda/lib/python3.5/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


counts =  [222902, 71422]
means =  [1.7412422311204827, 3.517923763056265]


/Users/rorywilliams/anaconda/lib/python3.5/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


...

/Users/rorywilliams/anaconda/lib/python3.5/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


counts =  [32579, 14963]
means =  [1.8121404528681793, 3.7240458923373194]


/Users/rorywilliams/anaconda/lib/python3.5/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


counts =  [49212]
means =  [1.9654858406042577]
counts =  [49167]
means =  [1.9554164581408802]


/Users/rorywilliams/anaconda/lib/python3.5/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [20]:
# Label dataframe with metadata_flow
plate1_peaks_labeled = select_add_metadata_flow(plate1_peaks_RW,'./20190204_T7diff_metadata_flow.csv',wells)
plate2_peaks_labeled = select_add_metadata_flow(plate2_peaks_RW,'./20190204_T7diff_metadata_flow.csv',wells)
plate3_peaks_labeled = select_add_metadata_flow(plate3_peaks_RW,'./20190204_T7diff_metadata_flow.csv',wells)
plate4_peaks_labeled = select_add_metadata_flow(plate4_peaks_RW,'./20190204_T7diff_metadata_flow.csv',wells)
plate5_peaks_labeled = select_add_metadata_flow(plate5_peaks_RW,'./20190204_T7diff_metadata_flow.csv',wells)
plate6_peaks_labeled = select_add_metadata_flow(plate6_peaks_RW,'./20190204_T7diff_metadata_flow.csv',wells2)
plate7_peaks_labeled = select_add_metadata_flow(plate7_peaks_RW,'./20190204_T7diff_metadata_flow.csv',wells2)
plate8_peaks_labeled = select_add_metadata_flow(plate8_peaks_RW,'./20190204_T7diff_metadata_flow.csv',wells2)


In [21]:
# Generate dataframe with peaks from all plates
plates = [plate1_peaks_labeled,plate2_peaks_labeled,plate3_peaks_labeled,plate4_peaks_labeled,
                  plate5_peaks_labeled,plate6_peaks_labeled,plate7_peaks_labeled,plate8_peaks_labeled]

for i, plate in enumerate(plates):
    if i==0:
        peaks_labeled = plate.copy()
        peaks_labeled['plate'] = 1
    else:
        p_add = plate.copy()
        p_add['plate'] = i + 1
        peaks_labeled = pd.concat([peaks_labeled,p_add])
        

In [22]:
# Use all detected on peaks to determine total fraction on
plate1_fracs = get_frac_on(plate1_peaks_RW,wells)
plate2_fracs = get_frac_on(plate2_peaks_RW,wells)
plate3_fracs = get_frac_on(plate3_peaks_RW,wells)
plate4_fracs = get_frac_on(plate4_peaks_RW,wells)
plate5_fracs = get_frac_on(plate5_peaks_RW,wells)
plate6_fracs = get_frac_on(plate6_peaks_RW,wells2)
plate7_fracs = get_frac_on(plate7_peaks_RW,wells2)
plate8_fracs = get_frac_on(plate8_peaks_RW,wells2)


In [23]:
# Add metadata_flow to dataframe
plate1_fracs_labeled = select_add_metadata_flow(plate1_fracs,'./20190204_T7diff_metadata_flow.csv',wells)
plate2_fracs_labeled = select_add_metadata_flow(plate2_fracs,'./20190204_T7diff_metadata_flow.csv',wells)
plate3_fracs_labeled = select_add_metadata_flow(plate3_fracs,'./20190204_T7diff_metadata_flow.csv',wells)
plate4_fracs_labeled = select_add_metadata_flow(plate4_fracs,'./20190204_T7diff_metadata_flow.csv',wells)
plate5_fracs_labeled = select_add_metadata_flow(plate5_fracs,'./20190204_T7diff_metadata_flow.csv',wells)
plate6_fracs_labeled = select_add_metadata_flow(plate6_fracs,'./20190204_T7diff_metadata_flow.csv',wells2)
plate7_fracs_labeled = select_add_metadata_flow(plate7_fracs,'./20190204_T7diff_metadata_flow.csv',wells2)
plate8_fracs_labeled = select_add_metadata_flow(plate8_fracs,'./20190204_T7diff_metadata_flow.csv',wells2)


In [24]:
# Generate dataframe with population fraction on for all plates
plates = [plate1_fracs_labeled,plate2_fracs_labeled,plate3_fracs_labeled,plate4_fracs_labeled,
                  plate5_fracs_labeled,plate6_fracs_labeled,plate7_fracs_labeled,plate8_fracs_labeled]

for i, plate in enumerate(plates):
    if i==0:
        fracs_labeled = plate.copy()
        fracs_labeled['plate'] = 1
    else:
        p_add = plate.copy()
        p_add['plate'] = i + 1
        fracs_labeled = pd.concat([fracs_labeled,p_add])
        
        

In [26]:
# Save dataframes
fracs_labeled.to_csv('./20190204_T7diff_fraction_on_summary.csv')
peaks_labeled.to_csv('./20190204_T7diff_all_peaks.csv')
